Work with SQLite in python

The SQLite python library is located in sqlite3 library, that's why in order to open a database we need to load the library. The documentation is here: https://docs.python.org/3/library/sqlite3.html

Opening a database that doesn't exist creates a new database file.

In [5]:
import sqlite3      #loading the library

db1conn = sqlite3.connect('F:/sqlite_databases/dabase.sqlite')

0. Data Frames (pandas)

We are interested in loading and storing data frames defined in pandas library more than in low level, thus, we need to load it.

In [6]:
import pandas as pd   # load the library

# build a query

query = "SELECT * FROM datable"

In [7]:
# now read into a dataframe df

df = pd.read_sql_query(query,db1conn)  # or pandas.read_sql (see below)
 
df.head()   # see the result

,Station.Name,Measurement.Timestamp,Air.Temperature,Wet.Bulb.Temperature,Humidity,Rain.Intensity,Interval.Rain,Total.Rain,Precipitation.Type,Wind.Direction,Wind.Speed,Maximum.Wind.Speed,Barometric.Pressure,Solar.Radiation,Heading,Battery.Life,Measurement.Timestamp.Label,Measurement.ID
0,Foster Weather Station,1.451714e+09,-1.56,None,62,None,0.0,None,None,269,2.1,3.0,995.9,0,None,15.2,01/02/2016 12:00 AM,FosterWeatherStation201601022400
1,Foster Weather Station,1.451718e+09,-2.28,None,63,None,0.0,None,None,282,2.4,2.6,995.6,0,None,15.2,01/02/2016 1:00 AM,FosterWeatherStation201601020100
2,Foster Weather Station,1.451722e+09,-2.89,None,65,None,0.0,None,None,262,4.0,4.7,996.3,0,None,14.8,01/02/2016 2:00 AM,FosterWeatherStation201601020200
3,Foster Weather Station,1.451725e+09,-3.06,None,64,None,0.0,None,None,278,2.3,3.2,995.9,0,None,14.8,01/02/2016 3:00 AM,FosterWeatherStation201601020300
4,Foster Weather Station,1.451729e+09,-3.17,None,64,None,0.0,None,None,304,2.2,2.5,995.9,0,None,15.2,01/02/2016 4:00 AM,FosterWeatherStation201601020400


In [9]:
db1conn.close()

For more details of pandas interaction see the McKinney book pages 174-175.
In particulare it mentions the following construct:

import pandas.io.sql as sql #load and alias the library
dataframe = sql.read_frame('select * from datable', db1conn)  #use the read_frame method

We need to look further in pandas documentation http://pandas.pydata.org/pandas-docs/stable/

In particular, in IO section: http://pandas.pydata.org/pandas-docs/stable/io.html
we see the read_sql function and the corresponding to_sql method for a dataframe.
The 'read_frame' is nowhere to be seen.

The description of pandas.io.sql is in this section:
http://pandas.pydata.org/pandas-docs/stable/io.html#io-sql

In particular it says that it's all in the read_sql_table, read_sql_query and read_sql functions now.

The key functions are:

read_sql_table(table_name, con[, schema, ...])	Read SQL database table into a DataFrame.

read_sql_query(sql, con[, index_col, ...])	Read SQL query into a DataFrame.

read_sql(sql, con[, index_col, ...])	Read SQL query or database table into a DataFrame.

DataFrame.to_sql(name, con[, flavor, ...])	Write records stored in a DataFrame to a SQL database.

Note: The function read_sql() is a convenience wrapper around read_sql_table() and read_sql_query() (and for backward compatibility) and will delegate to specific function depending on the provided input (database table name or sql query). Table names do not need to be quoted if they have special characters.

And the documentation clearly pushes you one level up to the 'sqlalchemy' library, and the corresponding dialects of SQL
http://docs.sqlalchemy.org/en/latest/dialects/index.html

The whole higher level library story starts here: http://www.sqlalchemy.org/

Writing pandas dataframe function is described here: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html#pandas.DataFrame.to_sql

2 Creating a table

In [3]:
import sqlite3      #loading the library

db1conn = sqlite3.connect('F:/sqlite_databases/dabase.sqlite') # establish a connection
c = db1conn.cursor() # create a cursor

c.execute('''CREATE TABLE stocks
             (date text, trans text, symbol text, qty real, price real)''') #create table
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)") #insert row

db1conn.commit() # commit the changes to the connection

db1conn.close() # close the connection

Works perfectly well.
So, basically 'execute' is the right verb to remember for data manipulation both in R and in Python.

3 Synthetic queries

They totally insist on using wildcards in queries and NEVER to use string algebra for making a text of a compound query programmatically. The examples are:

In [4]:
# fetching one line

import sqlite3      #loading the library

db1conn = sqlite3.connect('F:/sqlite_databases/dabase.sqlite') # establish a connection
c = db1conn.cursor() # create a cursor

t = ('RHAT',)  #assigning a value to a parameter
c.execute('SELECT * FROM stocks WHERE symbol=?', t) # substitute instead of the wildcard
print(c.fetchone())

db1conn.close() # close the connection

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


Works

In [12]:
# adding multiple lines

import sqlite3      #loading the library

db1conn = sqlite3.connect('F:/sqlite_databases/dabase.sqlite') # establish a connection
c = db1conn.cursor() # create a cursor

purchases = [('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'SELL', 'IBM', 500, 53.00),
            ]
c.executemany('INSERT INTO stocks VALUES (?,?,?,?,?)', purchases)

db1conn.commit() # commit the changes to the connection

db1conn.close() # close the connection

In [13]:
# retrieving multiple lines

import sqlite3      #loading the library

db1conn = sqlite3.connect('F:/sqlite_databases/dabase.sqlite') # establish a connection
c = db1conn.cursor() # create a cursor

c.execute('SELECT * FROM stocks ORDER BY price')

df = c.fetchall()

print(df)

db1conn.close() # close the connection

[('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14), ('2006-03-28', 'BUY', 'IBM', 1000.0, 45.0), ('2006-04-06', 'SELL', 'IBM', 500.0, 53.0), ('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)]
